In [1]:
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.optimizers import Adam
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [2]:
path = "/kaggle/input/dataset-chest/"

In [3]:
df_chest_filt = pd.read_pickle((path + "combined_chest_filtered.pkl"))

In [4]:
df_chest_filt.head()

,id,ACC_x,ACC_y,ACC_z,ECG,EMG,EDA,TEMP,RESP,label
214583,2.0,0.8914,-0.1102,-0.2576,0.030945,-0.003708,5.710983,29.083618,1.191711,1.0
214584,2.0,0.8926,-0.1086,-0.2544,0.033646,-0.014145,5.719376,29.122437,1.139832,1.0
214585,2.0,0.8930,-0.1094,-0.2580,0.033005,0.010208,5.706406,29.115234,1.141357,1.0
214586,2.0,0.8934,-0.1082,-0.2538,0.031815,0.012634,5.712509,29.126709,1.155090,1.0
214587,2.0,0.8930,-0.1096,-0.2570,0.030350,0.002060,5.727005,29.100861,1.133728,1.0


In [5]:
df_chest_filt.isnull().sum()

id       0
ACC_x    0
ACC_y    0
ACC_z    0
ECG      0
EMG      0
EDA      0
TEMP     0
RESP     0
label    0
dtype: int64

In [5]:
df_chest_filt.isna().any()

id       False
ACC_x    False
ACC_y    False
ACC_z    False
ECG      False
EMG      False
EDA      False
TEMP     False
RESP     False
label    False
dtype: bool

In [6]:
df_chest_filt.groupby(['id', 'label']).count()

ACC_x   ACC_y   ACC_z     ECG     EMG     EDA    TEMP    RESP
id   label                                                                
2.0  1.0    800800  800800  800800  800800  800800  800800  800800  800800
     2.0    430500  430500  430500  430500  430500  430500  430500  430500
     3.0    253400  253400  253400  253400  253400  253400  253400  253400
3.0  1.0    798000  798000  798000  798000  798000  798000  798000  798000
     2.0    447917  447917  447917  447917  447917  447917  447917  447917
     3.0    262500  262500  262500  262500  262500  262500  262500  262500
4.0  1.0    810601  810601  810601  810601  810601  810601  810601  810601
     2.0    444500  444500  444500  444500  444500  444500  444500  444500
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
5.0  1.0    838600  838600  838600  838600  838600  838600  838600  838600
     2.0    451500  451500  451500  451500  451500  451500  451500  451500
     3.0    261800  261800  261800  261800  261800  261800  261800  261800
6.0  1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    455000  455000  455000  455000  455000  455000  455000  455000
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
7.0  1.0    830200  830200  830200  830200  830200  830200  830200  830200
     2.0    448000  448000  448000  448000  448000  448000  448000  448000
     3.0    260401  260401  260401  260401  260401  260401  260401  260401
8.0  1.0    818300  818300  818300  818300  818300  818300  818300  818300
     2.0    469000  469000  469000  469000  469000  469000  469000  469000
     3.0    258999  258999  258999  258999  258999  258999  258999  258999
9.0  1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    451500  451500  451500  451500  451500  451500  451500  451500
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
10.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    507500  507500  507500  507500  507500  507500  507500  507500
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
11.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    476000  476000  476000  476000  476000  476000  476000  476000
     3.0    257600  257600  257600  257600  257600  257600  257600  257600
13.0 1.0    826001  826001  826001  826001  826001  826001  826001  826001
     2.0    464800  464800  464800  464800  464800  464800  464800  464800
     3.0    267400  267400  267400  267400  267400  267400  267400  267400
14.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    472500  472500  472500  472500  472500  472500  472500  472500
     3.0    260401  260401  260401  260401  260401  260401  260401  260401
15.0 1.0    822500  822500  822500  822500  822500  822500  822500  822500
     2.0    480200  480200  480200  480200  480200  480200  480200  480200
     3.0    260400  260400  260400  260400  260400  260400  260400  260400
16.0 1.0    826000  826000  826000  826000  826000  826000  826000  826000
     2.0    471101  471101  471101  471101  471101  471101  471101  471101
     3.0    257600  257600  257600  257600  257600  257600  257600  257600
17.0 1.0    826700  826700  826700  826700  826700  826700  826700  826700
     2.0    506100  506100  506100  506100  506100  506100  506100  506100
     3.0    260400  260400  260400  260400  260400  260400  260400  260400

In [4]:
# Extraer features y labels
X = df_chest_filt[['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'TEMP', 'RESP']].values
y = df_chest_filt['label'].values

In [5]:
y.dtype

dtype('float64')

In [5]:
# Primero se converte la columna de tipo int a tipo categórico
y = pd.Categorical(y)
y.dtype

CategoricalDtype(categories=[1.0, 2.0, 3.0], ordered=False)

# Clasificación binaria para los datos de pecho

In [6]:
# Crear un modelo CNN para clasificación binaria para los datos recopilados de pecho
sampling_rate = 700  # Hz
window_time = 5  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

# Crear segmentos con overlapping windows
segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)
# Clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = np.where((y_segments == 1) | (y_segments == 3), 0, 1)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=8, kernel_size=15, stride=2, pool_size=4, pool_stride=4)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3500, 8)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1743, 8)           968       
                                                                 
 max_pooling1d (MaxPooling1  (None, 435, 8)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3480)              0         
                                                                 
 dense (Dense)               (None, 32)                111392    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                             

In [8]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo
model_c_binary.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
332/332 [==============================] - 7s 17ms/step - loss: 0.3267 - accuracy: 0.8287 - val_loss: 0.2140 - val_accuracy: 0.9069
Epoch 2/20
332/332 [==============================] - 5s 16ms/step - loss: 0.1561 - accuracy: 0.9373 - val_loss: 0.1947 - val_accuracy: 0.9223
Epoch 3/20
332/332 [==============================] - 5s 16ms/step - loss: 0.1295 - accuracy: 0.9491 - val_loss: 0.2896 - val_accuracy: 0.9114
Epoch 4/20
332/332 [==============================] - 5s 16ms/step - loss: 0.1060 - accuracy: 0.9588 - val_loss: 0.1377 - val_accuracy: 0.9446
Epoch 5/20
332/332 [==============================] - 5s 16ms/step - loss: 0.0785 - accuracy: 0.9705 - val_loss: 0.1754 - val_accuracy: 0.9363
Epoch 6/20
332/332 [==============================] - 5s 16ms/step - loss: 0.0682 - accuracy: 0.9753 - val_loss: 0.1565 - val_accuracy: 0.9408
Epoch 7/20
332/332 [==============================] - 5s 16ms/step - loss: 0.0515 - accuracy: 0.9819 - val_loss: 0.1618 - val_accuracy: 0.9400

In [9]:
y_pred_binary = (model_c_binary.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

83/83 [==============================] - 1s 5ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9312
Precisión: 0.9574
Recall: 0.9295
Matriz de confusión:
[[1774   55]
 [  58  765]]


In [7]:
# Mejorar el modelo modificando algunos parámetros
sampling_rate = 700  # Hz
window_time = 1  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)
# Clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = np.where((y_segments == 1) | (y_segments == 3), 0, 1)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=8, kernel_size=15, stride=2, pool_size=4, pool_stride=4)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary_2 = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 700, 8)]          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 343, 8)            968       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 85, 8)             0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 680)               0         
                                                                 
 dense_3 (Dense)             (None, 32)                21792     
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                           

In [12]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo con nuevo parámetro
model_c_binary_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary_2.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary_2.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
1658/1658 [==============================] - 13s 7ms/step - loss: 0.1750 - accuracy: 0.9220 - val_loss: 0.0863 - val_accuracy: 0.9694
Epoch 2/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.0669 - accuracy: 0.9778 - val_loss: 0.0511 - val_accuracy: 0.9820
Epoch 3/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.0454 - accuracy: 0.9855 - val_loss: 0.0507 - val_accuracy: 0.9836
Epoch 4/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.0347 - accuracy: 0.9896 - val_loss: 0.0644 - val_accuracy: 0.9808
Epoch 5/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.0330 - accuracy: 0.9896 - val_loss: 0.0386 - val_accuracy: 0.9891
Epoch 6/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.0284 - accuracy: 0.9910 - val_loss: 0.0446 - val_accuracy: 0.9863
Epoch 7/20
1658/1658 [==============================] - 11s 7ms/step - loss: 0.0232 - accuracy: 0.9927 - val_loss: 0.0274 - val_ac

In [13]:
y_pred_binary = (model_c_binary_2.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 1s 3ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9893
Precisión: 0.9934
Recall: 0.9853
Matriz de confusión:
[[9162   27]
 [  60 4012]]


Ha mejorado los resultados por disminuir el window_time a 1s, ahora se cambia otro parámetro para seguir mejorando los resultados.

In [8]:
# Mejorar el modelo modificando algunos parámetros
sampling_rate = 700  # Hz
window_time = 1  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)

# Clasificar label 1(fase neutral) y 3(diversión) a un grupo(no estrés) y label 2(estrés) a otro
y_segments_binary = np.where((y_segments == 1) | (y_segments == 3), 0, 1)

# Redimensionar X_segments para adaptar a la dimensión de entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# Definir el input layer para todos los sensores
input_layer = Input(shape=(sequence_length, num_features))

# Función para crear un bloque convolucional de 1D 
def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary_3 = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 700, 8)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 343, 16)           1936      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 85, 16)            0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 1360)              0         
                                                                 
 dense_6 (Dense)             (None, 32)                43552     
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                           

In [15]:
# Normalización de datos
scaler = StandardScaler()
X_segments_normalized = scaler.fit_transform(X_segments_reshaped.reshape(-1, num_features)).reshape(X_segments_reshaped.shape)

# Dividir los datos para entrenamiento(80%) y para prueba(20%)
X_train, X_test, y_train, y_test = train_test_split(X_segments_normalized, y_segments_binary, test_size=0.2, random_state=42)

# Ejecutar el modelo con nuevos parámetros
model_c_binary_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary_3.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary_3.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
1658/1658 [==============================] - 14s 8ms/step - loss: 0.1420 - accuracy: 0.9421 - val_loss: 0.0705 - val_accuracy: 0.9796
Epoch 2/20
1658/1658 [==============================] - 12s 7ms/step - loss: 0.0557 - accuracy: 0.9829 - val_loss: 0.0405 - val_accuracy: 0.9871
Epoch 3/20
1658/1658 [==============================] - 14s 8ms/step - loss: 0.0410 - accuracy: 0.9877 - val_loss: 0.0375 - val_accuracy: 0.9902
Epoch 4/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.0330 - accuracy: 0.9901 - val_loss: 0.0342 - val_accuracy: 0.9912
Epoch 5/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.0288 - accuracy: 0.9914 - val_loss: 0.0451 - val_accuracy: 0.9870
Epoch 6/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.0216 - accuracy: 0.9935 - val_loss: 0.0333 - val_accuracy: 0.9908
Epoch 7/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.0228 - accuracy: 0.9931 - val_loss: 0.0473 - val_ac

In [16]:
y_pred_binary = (model_c_binary_3.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 1s 3ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9905
Precisión: 0.9942
Recall: 0.9889
Matriz de confusión:
[[9157   32]
 [  45 4027]]


Ha mejorado todavía un poco los resultados al aumentar el número de filtros a 16, ahora modifica otro parámetro para intentar a seguir mejorando.

In [9]:
# Mejorar el modelo añadir 2 capas dropout
x = Dense(32, activation='relu')(Flatten()(output))
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary_4 = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary_4.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 700, 8)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 343, 16)           1936      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 85, 16)            0         
 g1D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1360)              0         
                                                                 
 dense_9 (Dense)             (None, 32)                43552     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                           

In [18]:
# Ejecutar el modelo con nuevos parámetros
model_c_binary_4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary_4.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary_4.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
1658/1658 [==============================] - 15s 8ms/step - loss: 0.4023 - accuracy: 0.7544 - val_loss: 0.1861 - val_accuracy: 0.8743
Epoch 2/20
1658/1658 [==============================] - 14s 9ms/step - loss: 0.2816 - accuracy: 0.8317 - val_loss: 0.1203 - val_accuracy: 0.9349
Epoch 3/20
1658/1658 [==============================] - 14s 9ms/step - loss: 0.2384 - accuracy: 0.8674 - val_loss: 0.0856 - val_accuracy: 0.9577
Epoch 4/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.2029 - accuracy: 0.8939 - val_loss: 0.0832 - val_accuracy: 0.9621
Epoch 5/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.1798 - accuracy: 0.9189 - val_loss: 0.0790 - val_accuracy: 0.9605
Epoch 6/20
1658/1658 [==============================] - 13s 8ms/step - loss: 0.1542 - accuracy: 0.9335 - val_loss: 0.0698 - val_accuracy: 0.9630
Epoch 7/20
1658/1658 [==============================] - 12s 8ms/step - loss: 0.1372 - accuracy: 0.9412 - val_loss: 0.0614 - val_ac

Ha empeorando por añadir 2 capas dropout, esto modelo se tiene que descartar y volver a coger el modelo model_c_binary_3 para más modificaciones.

In [10]:
# Mejorar el modelo añadiendo otra capa de output
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

# Generar el modelo
model_c_binary_5 = Model(inputs=input_layer, outputs=output_layer)

# Imprimir por pantalla el resumen del modelo
model_c_binary_5.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 700, 8)]          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 343, 16)           1936      
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 85, 16)            0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 76, 32)            5152      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 38, 32)            0         
 g1D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 1216)              0   

In [26]:
# Ejecutar el modelo con nuevos parámetros
model_c_binary_5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary_5.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary_5.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/20
1658/1658 [==============================] - 19s 11ms/step - loss: 0.1152 - accuracy: 0.9539 - val_loss: 0.0523 - val_accuracy: 0.9839
Epoch 2/20
1658/1658 [==============================] - 16s 10ms/step - loss: 0.0385 - accuracy: 0.9881 - val_loss: 0.0435 - val_accuracy: 0.9885
Epoch 3/20
1658/1658 [==============================] - 16s 9ms/step - loss: 0.0294 - accuracy: 0.9909 - val_loss: 0.0216 - val_accuracy: 0.9947
Epoch 4/20
1658/1658 [==============================] - 16s 9ms/step - loss: 0.0238 - accuracy: 0.9928 - val_loss: 0.0207 - val_accuracy: 0.9949
Epoch 5/20
1658/1658 [==============================] - 16s 10ms/step - loss: 0.0207 - accuracy: 0.9939 - val_loss: 0.0205 - val_accuracy: 0.9956
Epoch 6/20
1658/1658 [==============================] - 17s 10ms/step - loss: 0.0158 - accuracy: 0.9956 - val_loss: 0.0274 - val_accuracy: 0.9915
Epoch 7/20
1658/1658 [==============================] - 17s 10ms/step - loss: 0.0164 - accuracy: 0.9958 - val_loss: 0.0159 - v

In [27]:
y_pred_binary = (model_c_binary_5.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 2s 4ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9952
Precisión: 0.9971
Recall: 0.9944
Matriz de confusión:
[[9173   16]
 [  23 4049]]


Con otra capa de output ha mejorado los resultados, ahora se parte de este mismo modelo mode_c_binary_5 modificando otro parámetro learning rate a 0,01 para ver si se puede mejorar más.

In [28]:
# Modificar el modelo model_c_binary_5 con nuevo learning rate=0,01
optimizer = Adam(lr=0.01)
model_c_binary_5.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary_5.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary_5.evaluate(X_test, y_test)


Epoch 1/20
1658/1658 [==============================] - 19s 11ms/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 0.0160 - val_accuracy: 0.9966
Epoch 2/20
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0057 - accuracy: 0.9983 - val_loss: 0.0176 - val_accuracy: 0.9967
Epoch 3/20
1658/1658 [==============================] - 17s 10ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 0.0197 - val_accuracy: 0.9961
Epoch 4/20
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0048 - accuracy: 0.9984 - val_loss: 0.0110 - val_accuracy: 0.9977
Epoch 5/20
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0053 - accuracy: 0.9987 - val_loss: 0.0166 - val_accuracy: 0.9975
Epoch 6/20
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0051 - accuracy: 0.9984 - val_loss: 0.0207 - val_accuracy: 0.9954
Epoch 7/20
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0050 - accuracy: 0.9983 - val_loss: 0.0095 -

In [29]:
y_pred_binary = (model_c_binary_5.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 2s 4ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9957
Precisión: 0.9974
Recall: 0.9980
Matriz de confusión:
[[9162   27]
 [   8 4064]]


Los resultados han mejorado un poco más cambiando el learning rate a 0,01, estos valores son bastantes buenos, ahora se prueba con más número de epochs para seguir mejorando.

In [15]:
# Ejecutar el modelo model_c_binary3 con número de epochs de 50
optimizer = Adam(lr=0.01)
model_c_binary_5.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model_c_binary_5.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

# Evaluar la precisión con los datos de prueba
loss, accuracy = model_c_binary_5.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación binaria para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/40
1658/1658 [==============================] - 19s 11ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 0.0327 - val_accuracy: 0.9968
Epoch 2/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0030 - accuracy: 0.9992 - val_loss: 0.0245 - val_accuracy: 0.9977
Epoch 3/40
1658/1658 [==============================] - 17s 11ms/step - loss: 0.0015 - accuracy: 0.9997 - val_loss: 0.0207 - val_accuracy: 0.9970
Epoch 4/40
1658/1658 [==============================] - 18s 11ms/step - loss: 8.3912e-04 - accuracy: 0.9998 - val_loss: 0.0287 - val_accuracy: 0.9978
Epoch 5/40
1658/1658 [==============================] - 18s 11ms/step - loss: 1.4270e-05 - accuracy: 1.0000 - val_loss: 0.0318 - val_accuracy: 0.9980
Epoch 6/40
1658/1658 [==============================] - 17s 10ms/step - loss: 0.0048 - accuracy: 0.9991 - val_loss: 0.0225 - val_accuracy: 0.9973
Epoch 7/40
1658/1658 [==============================] - 19s 11ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 

In [16]:
y_pred_binary = (model_c_binary_5.predict(X_test) > 0.5).astype(int)
y_true_binary = y_test 
# Calcular F1-score, precisión y recall
f1 = f1_score(y_true_binary, y_pred_binary)
precision = precision_score(y_true_binary, y_pred_binary)
recall = recall_score(y_true_binary, y_pred_binary)

# Imprimir los reesultados
print("Resultados del modelo de clasificación binaria para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}') # Recall es la Tasa de verdaderos positivos

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_binary, y_pred_binary)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 2s 5ms/step
Resultados del modelo de clasificación binaria para datos de pecho: 
F1-score: 0.9968
Precisión: 0.9980
Recall: 0.9971
Matriz de confusión:
[[9175   14]
 [  12 4060]]


Han conseguido muy buenos resultados. Debido a la capacidad limitada de la memoria RAM, no se comprueba con más nº de epochs.

Los resultados son bastantes buenos, ya no tiene mucho sentido modificar más parámetros. Como resumen, los parámetros utilizados son:

sampling rate: 700Hz

window size: 1s

sequence length: 700*1

step size=sequence length//2

nº de capas de output y los parámetros son:
* capa 1 de output: = filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4
* capa 2 de output: filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2

optimizer: adam con learning rate=0,01

nº epochs: 40

# Clasificación de tres estados afectivos para los datos de pecho

In [31]:
# Generar un modelo CNN para la clasificaión de tres estados para los datos de pecho
# Modificar el modelo anterior para adaptar a la nueva función
sampling_rate = 700  # Hz
window_time = 1  # segundos
sequence_length = sampling_rate * window_time
step_size = sequence_length // 2

segments = []
labels = []
for i in range(0, len(X) - sequence_length, step_size):
    segment = X[i:i + sequence_length]
    label = y[i + sequence_length - 1]  
    segments.append(segment)
    labels.append(label)

# Convertir a vector 
X_segments = np.array(segments)
y_segments = np.array(labels)

# Redimensionar X_segments para la entrada para LSTM
num_features = X_segments.shape[2]
X_segments_reshaped = X_segments.reshape((X_segments.shape[0], X_segments.shape[1], num_features))

# One hot encoding, num_classes = 3
y_segments = y_segments - 1
y_segments_one_hot = to_categorical(y_segments, num_classes=3) 

# Dividir datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_segments_reshaped, y_segments_one_hot, test_size=0.2, random_state=42)

input_layer = Input(shape=(sequence_length, num_features))

def conv_block(x, filters, kernel_size, stride, pool_size, pool_stride):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride, activation='relu')(x)
    x = MaxPooling1D(pool_size=pool_size, strides=pool_stride)(x)
    return x

# Aplicar el bloque convolucional al input
output = conv_block(input_layer, filters=16, kernel_size=15, stride=2, pool_size=4, pool_stride=4)
output = conv_block(output, filters=32, kernel_size=10, stride=1, pool_size=2, pool_stride=2)

# Conexión completa entre layers
x = Dense(32, activation='relu')(Flatten()(output))
x = Dense(16, activation='relu')(x)

# Output layer para 3 clases, donde 3 es el num_classes
output_layer = Dense(3, activation='softmax')(x)

# Generar el modelo para clasificación de 3 estados
model_c_three = Model(inputs=input_layer, outputs=output_layer)

model_c_three.summary()
optimizer = Adam(lr=0.01)
model_c_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_c_three.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_c_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de pecho: {accuracy * 100:.2f}%')


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 700, 8)]          0         
                                                                 
 conv1d_9 (Conv1D)           (None, 343, 16)           1936      
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 85, 16)            0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 76, 32)            5152      
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 38, 32)            0         
 ng1D)                                                           
                                                                 
 flatten_7 (Flatten)         (None, 1216)              0   

In [34]:
y_pred_three = np.argmax(model_c_three.predict(X_test), axis=1)
y_true_three = np.argmax(y_test, axis=1)

f1 = f1_score(y_true_three, y_pred_three, average='weighted')
precision = precision_score(y_true_three, y_pred_three, average='weighted')
recall = recall_score(y_true_three, y_pred_three, average='weighted')

# Imprimir los reesultados
print("Resultados del modelo de clasificación de tres estados para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_three, y_pred_three)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 2s 4ms/step
Resultados del modelo de clasificación de tres estados para datos de pecho: 
F1-score: 0.9793
Precisión: 0.9793
Recall: 0.9793
Matriz de confusión:
[[6912   28   65]
 [  16 4030   26]
 [  69   70 2045]]


Intenta mejorar el resultado modificando algunos parámetros

In [36]:
optimizer = Adam(lr=0.001)
model_c_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_c_three.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model_c_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de pecho: {accuracy * 100:.2f}%')


Epoch 1/40
1658/1658 [==============================] - 19s 11ms/step - loss: 0.0438 - accuracy: 0.9863 - val_loss: 0.0562 - val_accuracy: 0.9868
Epoch 2/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0433 - accuracy: 0.9863 - val_loss: 0.0621 - val_accuracy: 0.9825
Epoch 3/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0388 - accuracy: 0.9872 - val_loss: 0.0593 - val_accuracy: 0.9854
Epoch 4/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0373 - accuracy: 0.9885 - val_loss: 0.0684 - val_accuracy: 0.9859
Epoch 5/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0389 - accuracy: 0.9879 - val_loss: 0.0620 - val_accuracy: 0.9848
Epoch 6/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0398 - accuracy: 0.9867 - val_loss: 0.0545 - val_accuracy: 0.9877
Epoch 7/40
1658/1658 [==============================] - 18s 11ms/step - loss: 0.0392 - accuracy: 0.9877 - val_loss: 0.1197 -

In [38]:
y_pred_three = np.argmax(model_c_three.predict(X_test), axis=1)
y_true_three = np.argmax(y_test, axis=1)

f1 = f1_score(y_true_three, y_pred_three, average='weighted')
precision = precision_score(y_true_three, y_pred_three, average='weighted')
recall = recall_score(y_true_three, y_pred_three, average='weighted')

# Imprimir los reesultados
print("Resultados del modelo de clasificación de tres estados para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_three, y_pred_three)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 2s 4ms/step
Resultados del modelo de clasificación de tres estados para datos de pecho: 
F1-score: 0.9861
Precisión: 0.9860
Recall: 0.9860
Matriz de confusión:
[[6965    7   33]
 [  31 3989   52]
 [  46   16 2122]]


Se ha mejorado con lr=0,001, ahora prueba con batch size=16 para ver si puede mejorar algo más.

In [40]:
optimizer = Adam(lr=0.001)
model_c_three.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_c_three.fit(X_train, y_train, epochs=40, batch_size=16, validation_data=(X_test, y_test))

loss, accuracy = model_c_three.evaluate(X_test, y_test)
print(f'Precisión del modelo de clasificación de tres estados para datos de pecho: {accuracy * 100:.2f}%')

Epoch 1/40
3316/3316 [==============================] - 28s 8ms/step - loss: 0.0445 - accuracy: 0.9880 - val_loss: 0.0718 - val_accuracy: 0.9873
Epoch 2/40
3316/3316 [==============================] - 27s 8ms/step - loss: 0.0445 - accuracy: 0.9865 - val_loss: 0.0726 - val_accuracy: 0.9839
Epoch 3/40
3316/3316 [==============================] - 26s 8ms/step - loss: 0.0487 - accuracy: 0.9859 - val_loss: 0.0615 - val_accuracy: 0.9879
Epoch 4/40
3316/3316 [==============================] - 26s 8ms/step - loss: 0.0443 - accuracy: 0.9877 - val_loss: 0.0680 - val_accuracy: 0.9856
Epoch 5/40
3316/3316 [==============================] - 26s 8ms/step - loss: 0.0431 - accuracy: 0.9870 - val_loss: 0.0952 - val_accuracy: 0.9791
Epoch 6/40
3316/3316 [==============================] - 27s 8ms/step - loss: 0.0443 - accuracy: 0.9869 - val_loss: 0.0774 - val_accuracy: 0.9816
Epoch 7/40
3316/3316 [==============================] - 27s 8ms/step - loss: 0.0406 - accuracy: 0.9886 - val_loss: 0.0718 - val_ac

In [41]:
y_pred_three = np.argmax(model_c_three.predict(X_test), axis=1)
y_true_three = np.argmax(y_test, axis=1)

f1 = f1_score(y_true_three, y_pred_three, average='weighted')
precision = precision_score(y_true_three, y_pred_three, average='weighted')
recall = recall_score(y_true_three, y_pred_three, average='weighted')

# Imprimir los reesultados
print("Resultados del modelo de clasificación de tres estados para datos de pecho: ")
print(f'F1-score: {f1:.4f}')
print(f'Precisión: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

# Imprimir la matriz de confusión
conf_matrix = confusion_matrix(y_true_three, y_pred_three)
print('Matriz de confusión:')
print(conf_matrix)

415/415 [==============================] - 2s 5ms/step
Resultados del modelo de clasificación de tres estados para datos de pecho: 
F1-score: 0.9818
Precisión: 0.9818
Recall: 0.9818
Matriz de confusión:
[[6906   19   80]
 [  33 4007   32]
 [  19   59 2106]]


Los resultados han empeorado comparando con el anterior, pero hay que tener en cuenta que esa precisión que consigue es bastante alta, es normal que no puede alcanzar más. Por lo tanto, se queda con los resultados del modelo anterior, es decir, con una precisión de 98,60%.